# Import libraries

In [1]:
import torch
import json
from torch import nn
from tool import utils, preprocess, architecture, analysis
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import time
import os

# Before the analysis ...

## Get user input

In [2]:
ANALYSIS_NAME = ... # Specify the folder name here (ex ANALYSIS_NAME = 'Model1_Preproc') ...
ANALYSIS_DIR_PATH = os.path.join('./RUN', ANALYSIS_NAME)
params = analysis.get_params(ANALYSIS_DIR_PATH)
params

{'ARCHITECTURE': '1', 'TREATMENT': 'no'}

## Build test data loader

In [4]:
test_loader, img_size = analysis.get_test_loader(params)

Compose(
    ToTensor()
)


## Build and load CNN model

In [5]:
model = analysis.build_load_model(params, ANALYSIS_DIR_PATH, img_size)

In [6]:
model

CNN_Model1(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=307200, out_features=18, bias=True)
)

# Analyses starts here ...

## Make predictions 

In [7]:
since = time.time()

true_label, predict = analysis.predict(model, test_loader)

time_elapsed = time.time() - since
print('Predictions on {} test images takes {:.0f}m {:.0f}s'.format(
    len(test_loader.dataset), time_elapsed // 60, time_elapsed % 60))

Predictions on 852 test images takes 1m 46s


## Accuracy

In [8]:
np.mean(true_label == predict) # 0.87

0.8732394366197183

## Confusion matrix

In [9]:
con = confusion_matrix(true_label, predict)
con_df = pd.DataFrame(con)
con_df.columns = test_loader.dataset.classes
con_df.index = test_loader.dataset.classes

In [10]:
con_df

,L_1,L_2,L_3,L_4,L_5,L_6,L_7,L_8,R_1,R_2,R_3,R_4,R_5,R_6,R_7,R_8,S_L,S_R
L_1,13,5,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
L_2,2,54,4,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
L_3,0,5,46,3,0,1,1,0,0,2,0,0,0,0,0,0,0,0
L_4,1,1,1,71,5,0,0,0,1,0,0,0,0,0,0,0,1,0
L_5,1,0,0,0,43,5,0,0,0,0,0,0,0,0,0,0,0,0
L_6,0,0,0,1,4,26,0,0,0,0,0,0,0,0,0,0,0,0
L_7,0,0,0,0,0,2,28,1,0,0,0,0,0,0,0,0,0,0
L_8,0,0,0,0,0,0,3,61,0,0,0,0,0,0,0,0,0,0
R_1,0,0,0,0,0,0,0,0,12,1,0,0,0,0,0,0,0,2
R_2,0,0,0,0,0,0,0,0,1,39,4,6,0,0,0,0,0,0


## Training logs

In [11]:
history = pd.read_csv(os.path.join(ANALYSIS_DIR_PATH, 'training_logger.csv'))

In [12]:
history

,Unnamed: 0,train_loss,train_acc,val_loss,val_acc
0,0,1.653174,0.485021,1.017999,0.681818
1,1,0.848606,0.730155,0.778506,0.737762
2,2,0.538581,0.826810,0.585884,0.816434
3,3,0.377355,0.876011,0.523267,0.850524
4,4,0.301172,0.898098,0.508123,0.869755
5,5,0.234411,0.919528,0.478046,0.872378
6,6,0.194746,0.935491,0.505857,0.873252
7,7,0.118676,0.962825,0.457940,0.888986
8,8,0.090390,0.975508,0.458969,0.890734
9,9,0.079854,0.978351,0.462041,0.894231
